In [1]:
import os

import pyttsx3
from espeakng import ESpeakNG
from gtts import gTTS

import abox
from config import *

def ttsGTTS(text: str):
    """
     current limit can be seen here https://cloud.google.com/text-to-speech/quotas
     see also https://github.com/pndurette/gTTS/issues/117#issuecomment-458634045
    :param text:
    """
    tts = gTTS(text, lang='de')
    tts.save(os.path.join(BASE_DIR, 'data_generation', "gTTS.mp3"))


# TODO not sure about the quality of espeak/espeak-ng
#  could be usefully to generate data with different pitch,speed volume etc.

# requires espeak(all) , nsss(mac) or sapi5(windows) to be installed locally
def ttsPyttsx3(text: str):
    engine = pyttsx3.init()
    engine.setProperty('voice', 'german')
    # rate: int = engine.getProperty('rate')
    # engine.setProperty('rate', rate - 0)
    # volume: int = engine.getProperty('volume')
    # engine.setProperty('volume', volume - 0.0)
    print("save_to_file")
    engine.save_to_file(text, os.path.join(BASE_DIR, 'data_generation', "pyttsx3.mp3"))
    print("runAndWait")
    engine.runAndWait()
    print("finished?")


# requires espeak-ng  to be installed locally
def ttsEsng(text: str):
    esng = ESpeakNG()
    esng.voice = 'german'
    esng.voice = 'German'
    # esng.pitch = 80
    # esng.speed = 120
    # esng.volume = 200
    #for v in esng.voices:
        #print(v)
    wavs = esng.synth_wav(text)
    if wavs:
        print("present")
        with open(os.path.join(BASE_DIR, 'data_generation', "esng.wav"), 'wb') as output:
            output.write(wavs)
    else:
        print("nope")
    # wav = wave.open(StringIO.StringIO(wavs))
    # print(wav.getnchannels(), wav.getframerate(), wav.getnframes())


# TODO https://acapela-box.com/AcaBox/index.php does not serve an api.
#  For a cloud base api see https://www.acapela-group.com/solutions/acapela-vaas/
# TODO not sure about this implementation -> needs python-pympv or other library that provides codecs
def ttsAbox(text: str):
    box = abox.Abox(output=os.path.join(BASE_DIR, 'data_generation', "abox.ogg"), voice="willbadguy22k")
    url = box.query(text)
    ret = abox.ab_download(url, box.option_l)
    if not ret:
        print("Aborting")


def googleApi():
    # TODO implement
    print("not implemented")


def bingApi():
    # TODO implement
    print("not implemented")


def mozillaTss():
    # see https://github.com/mozilla/TTS
    # TODO maybe some of the data of the open source datasets might be useful instead?
    #  https://www.caito.de/2019/01/the-m-ailabs-speech-dataset/
    print("not implemented")


def run(text: str):
    # ttsGTTS(text)
    # ttsPyttsx3(text)
    ttsEsng(text)
    # acapela(text)
    # ttsAbox(text)


run("Hallo Welt das hier ist ein deutscher Text gesprochen von einem Programm.")


present


In [14]:
# based on https://cloud.google.com/text-to-speech/docs/reference/libraries#cloud-console
# for pricing see https://cloud.google.com/text-to-speech/pricing
# for languages/voices see https://cloud.google.com/text-to-speech/docs/voices

import os
from google.oauth2 import service_account
from config import *
from google.cloud import texttospeech

os.makedirs(name=os.path.join(BASE_DIR, "data_generation", "google_cloud"),exist_ok=True)

# Instantiates a client
credentials = service_account.Credentials.from_service_account_file(GOOGLE_APPLICATION_CREDENTIALS)
client = texttospeech.TextToSpeechClient(credentials=credentials)

# Set the text input to be synthesized
text="Hallo Welt das hier ist ein deutscher Text gesprochen von einem Programm."
synthesis_input = texttospeech.types.SynthesisInput(text=text)

# Build the voice request, select the language and voice
# Standard Female
voice = texttospeech.types.VoiceSelectionParams(language_code="de-DE",name="de-DE-Standard-A")
# Standard Male
voice = texttospeech.types.VoiceSelectionParams(language_code="de-DE",name="de-DE-Standard-B")
# Neutral Ssml => seems to default to de-DE-Standard-A
voice = texttospeech.types.VoiceSelectionParams(language_code="de-DE",ssml_gender=texttospeech.enums.SsmlVoiceGender.NEUTRAL)
# Select the type of audio file you want returned
audio_config = texttospeech.types.AudioConfig(audio_encoding=texttospeech.enums.AudioEncoding.MP3)
# Perform the text-to-speech request on the text input with the selected voice parameters and audio file type
response = client.synthesize_speech(synthesis_input, voice, audio_config)

# The response's audio_content is binary.
with open(os.path.join(BASE_DIR, "data_generation", "google_cloud","google_cloud_n.mp3"), 'wb') as out:
    # Write the response to the output file.
    out.write(response.audio_content)
    print('Audio content written to file')

Audio content written to file


In [11]:
# based on https://azure.microsoft.com/en-us/pricing/details/cognitive-services/speech-services/
# for pricing see https://azure.microsoft.com/en-us/pricing/details/cognitive-services/speech-services/
# for voices see https://docs.microsoft.com/en-us/azure/cognitive-services/speech-service/language-support#standard-voices

# debug
# curl -v -X POST "https://YOUR_REGION.api.cognitive.microsoft.com/sts/v1.0/issueToken" -H "Ocp-Apim-Subscription-Key: YOUR_SUBSCRIPTION_KEY" -H "Content-type: application/x-www-form-urlencoded" -H "Content-Length: 0"
# may be related to https://github.com/Azure-Samples/cognitive-services-speech-sdk/issues/365

# FIXME use rest api in case the connection still fails https://docs.microsoft.com/en-us/azure/cognitive-services/speech-service/rest-speech-to-text
import os
import azure.cognitiveservices.speech as speechsdk
from config import *

os.makedirs(name=os.path.join(BASE_DIR, "data_generation", "azure_cloud"),exist_ok=True)

# Replace with your own subscription key and region identifier from here: https://aka.ms/speech/sdkregion
speech_config = speechsdk.SpeechConfig(subscription=AZURE_APPLICATION_KEY, region=AZURE_APPLICATION_REGION)

# Creates an audio configuration that points to an audio file.
# Replace with your own audio filename.
audio_filename = os.path.join(BASE_DIR, "data_generation", "azure_cloud","helloworld.mp3")
audio_output = speechsdk.audio.AudioOutputConfig(filename=audio_filename)

# Creates a synthesizer with the given settings
speech_synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config, audio_config=audio_output)

# Synthesizes the text to speech.
# Replace with your own text.
text = "Hello world!"
result = speech_synthesizer.speak_text_async(text).get()

# Checks result.
if result.reason == speechsdk.ResultReason.SynthesizingAudioCompleted:
    print("Speech synthesized to [{}] for text [{}]".format(audio_filename, text))
elif result.reason == speechsdk.ResultReason.Canceled:
    cancellation_details = result.cancellation_details
    print("Speech synthesis canceled: {}".format(cancellation_details.reason))
    if cancellation_details.reason == speechsdk.CancellationReason.Error:
        if cancellation_details.error_details:
            print("Error details: {}".format(cancellation_details.error_details))
            print(cancellation_details)
    print("Did you update the subscription info?")

Speech synthesis canceled: CancellationReason.Error
Error details: Connection failed (no connection to the remote host). Internal error: 1. Error details: 2460. Please check network connection, firewall setting, and the region name used to create speech factory. USP state: 2. Received audio size: 0bytes.
SpeechSynthesisCancellationDetails(reason=CancellationReason.Error, error_details="Connection failed (no connection to the remote host). Internal error: 1. Error details: 2460. Please check network connection, firewall setting, and the region name used to create speech factory. USP state: 2. Received audio size: 0bytes.")
Did you update the subscription info?
